# From updated activated T cells experiments 13.6.22

In [1]:
lib = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'
import sys
sys.path.append(lib)
from utilities.package_importing import *
import scipy.stats as stats
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt
import xgboost as xgb
from utilities.smart_seq_dataset import *
from utilities.smart_seq_dataset import *

### Go through all exp

In [4]:
total_scores = []

ALL_EXP_FOLDER = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy\XGBoost_facilities\Experiments\activated_t_cells'
for exp in os.listdir(ALL_EXP_FOLDER):
    exp_model_path = join(ALL_EXP_FOLDER, exp, r'Enhanced_XGboost_Model.pkl')
    enhanced_model = pickle.load(open(exp_model_path, "rb"))
    
    
    for layer in enhanced_model.model_layers:
        total_scores.append(layer[0].get_score(importance_type='gain'))
    

In [5]:
scores_dic = {}
for scores_xgbbost in total_scores:
    for gene, gene_score in scores_xgbbost.items():
        scores_dic[gene] = scores_dic.get(gene, [])+[gene_score]

In [6]:
score_df = pd.DataFrame(columns=['gene', 'avg_score', 'usage'])
for gene, gene_scores in scores_dic.items():
    score_df = score_df.append(pd.DataFrame([[gene, np.mean(gene_scores), len(gene_scores)]], columns=score_df.columns))

In [8]:
pd.set_option('display.max_rows', 30)

score_df.sort_values(['avg_score'], ascending=False).iloc[:20]#.gene.tolist()

,gene,avg_score,usage


# DEBUG

In [ ]:
EXP_PATH = r'C:\Users\KerenYlab\Desktop\Technion studies\courses\Cancer Bioinformatics - 066529\Final project\submission_CB\Results\All Experiments outputs\exp_44'
XGBOOST_FILE_NAME = r'Enhanced_XGboost_Model.pkl'

enhanced_model = pickle.load(open(join(EXP_PATH, XGBOOST_FILE_NAME), "rb"))
xgb_model = enhanced_model.model_layers[0][0]

In [ ]:
EXP_PATH = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy\XGBoost_facilities\Experiments\dummy'
XGBOOST_FILE_NAME = r'Enhanced_XGboost_Model.pkl'

enhanced_model = pickle.load(open(join(EXP_PATH, XGBOOST_FILE_NAME), "rb"))
shaply_object = enhanced_model.model_layers[0][3]
explainer = shaply_object[0]
shap_values = shaply_object[1]

In [ ]:
CONFIG_NAME = r'config.yaml'

with open(join(EXP_PATH, CONFIG_NAME), 'r') as f:
    config = yaml.safe_load(f)

In [ ]:
split_data_path = config['DATASET']['split_data_path']
pickle_path = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy\XGBoost_facilities\data\smartseq_data_4k_genes.p'

cells, gene_names, patients_information = pickle.load(open(pickle_path, "rb"))
# Builds datasets

whole_rna_seq_dataset = RNAseq_Dataset(cells, patients_information, gene_names)
train_patients_names = pickle.load(open(split_data_path, "rb"))
train_dataset, test_dataset = whole_rna_seq_dataset.split_by_patient_names(train_patients_names)